In [7]:
devtools::install_github("NPSDC/beaveR", ref = "dev")
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(beaveR))
clustFile <- "../brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/terminus/no_threshold_meaninf_0/group_nwk.txt"
quantDir <- file.path("../brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/out_sal") 
samples <- as.vector(outer(c(1:6), c(1,2), function(x,y) paste(x,y,sep='_')))
quantFiles <- file.path(quantDir, samples, 'quant.sf')
coldata <- data.frame(files=quantFiles, names=samples, condition=factor(rep(1:2, each=6)))
# tse <- buildTSE(treeTermFile = clustFile, coldata = coldata)

Skipping install of 'beaveR' from a github remote, the SHA1 (55f74bc0) has not changed since last install.
  Use `force = TRUE` to force installation

reading tree

importing quantifications

reading in files with read_tsv

1 
2 
3 
4 
5 
6 
7 
8 
9 
10 
11 
12 


found matching transcriptome:
[ GENCODE - Homo sapiens - release 26 ]

loading existing TxDb created: 2022-08-19 22:12:59

Loading required package: GenomicFeatures

Loading required package: AnnotationDbi

loading existing transcript ranges created: 2022-08-19 22:13:01

Missing txps 3621

Aggregation Started

Aggregation Ended



In [2]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
saveDir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
# save(tse, file=file.path(saveDir, "tseMean.RData"))
load(file.path(saveDir, "tseMean.RData"))

In [9]:
temp <- tse
tree <- rowTree(tse)
l <- length(tree$tip)

In [4]:
load("environment/brain_sim_nodtu/mode=gc_bias/simulate.rda")
suppressPackageStartupMessages(library(TreeSummarizedExperiment))
suppressPackageStartupMessages(library(ape))
tree <- rowTree(tse)
l <- length(tree$tip)
missingTxps <- setdiff(rownames(tse)[1:l], rownames(sim.counts.mat))
dts <- Reduce(union, c(names(iso.dge)[iso.dge], names(iso.dte)[iso.dte], names(iso.dte.only)[iso.dte.only], 
                       names(iso.dtu)[iso.dtu]))
dts <- intersect(dts, rownames(sim.counts.mat))
sim.counts.mat <- rbind(sim.counts.mat, matrix(0, nrow = length(missingTxps), ncol = ncol(sim.counts.mat),
                                               dimnames = list(missingTxps, colnames(sim.counts.mat))))
sim.counts.mat <- sim.counts.mat[rownames(tse)[1:l],]
dts <- intersect(dts, rownames(sim.counts.mat))

innNodes <- l+1:tree$Nnode
aggCountsNodes <- computeAggNodesU(tree, c(1:length(tree$tip),innNodes), sim.counts.mat)
logFCNodes <- ifelse(rowSums(aggCountsNodes)==0, 0, log2(aggCountsNodes[,2]+1)-log2(aggCountsNodes[,1]+1))
names(logFCNodes)[(l+1):length(logFCNodes)] <- paste("Node", names(logFCNodes)[(l+1):length(logFCNodes)], sep="")
rootFC <- abs(logFCNodes[l+1])


In [14]:
length(dts)
summary(abs(logFCNodes[dts]))
print(mean(abs(logFCNodes[dts]) >= rootFC))
rootFC
mcols(tse)[["meanInfRV"]][l+1]

[1] 8799

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
 0.8858  1.5380  1.9329  1.8816  2.2746  2.5824 

[1] 1


Node138760 
0.03264375

Node138760 
      0.01

In [10]:
metadata(tse)[["sf"]]

NULL

#### Run Swish

In [11]:
set.seed(1)
y <- fishpond::scaleInfReps(temp[1:l,])
y <- labelKeep(y)
y <- swish(y, x="condition")

In [13]:
detNodes <- list()
detNodes[["Swish"]] <- lapply(c(0.01, 0.05, 0.1), function(x) which(mcols(y)[,"qvalue"] <= x ))

negNodes <- list()
negNodes[["Swish"]] <- lapply(detNodes[["Swish"]], function(nodes) setdiff(seq(l), 
                                                          unlist(Descendants(tree, nodes))))

#### Min meanInfRV and Height

In [15]:
lambda <- c(0.05, 0.1, 1, 5, 10)

In [16]:
cuts <- list()
for(i in lambda) {
    descSize <- sapply(Descendants(tree, seq(nrow(tse))), length)
    metric <- (mcols(tse)[['meanInfRV']] + node.depth(tree, 2)*i) *descSize
    objS <- solveForOptimalCut(tse, metVec = metric, type = 'min')
    gammaN <- paste("gamma",i,sep="=")
    cuts[[gammaN]] <- objS[["cut"]]
}
names(cuts) <- paste("irv", "height", lambda, sep = "_") 
sapply(cuts,length)

irv_height_0.05  irv_height_0.1    irv_height_1    irv_height_5   irv_height_10 
          78507           89433          120525          133990          136746

In [17]:
for(i in seq_along(lambda)) {
     ihname <- paste("irv", "height", lambda[i], sep = "_") 
     y <- temp[sort(cuts[[ihname]]),]
     y <- fishpond::scaleInfReps(y)
     y <- labelKeep(y)
     set.seed(1)
     y <- swish(y, x="condition")
#      dim(y)
     detNodes[[ihname]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
#         txps <- match(nodes[grep("ENST", nodes)], rownames(tse))
#         iNodes <- match(nodes[grep("Node", nodes)], rownames(tse))
         match(nodes, names(logFCNodes))
        
#         c(txps,iNodes)
    })
    negNodes[[ihname]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })
}

In [26]:
lambda[i]

[1] 10

#### Max Weighted Log Fold Change

In [19]:
getLog2FC <- function(tse, condition, pc=5, array=FALSE) {
  infRepIdx <- grep("infRep", assayNames(tse))
  infReps <- assays(tse)[infRepIdx]
  infRepsArray <- abind::abind(as.list(infReps), along=3)
  dims <- dim(infRepsArray)
    
  cond1 <- condition == levels(condition)[1]
  cond2 <- condition == levels(condition)[2]

  diffs <- matrix(nrow=dims[1],ncol=dims[3])
  for (k in seq_len(dims[3])) {
    diffs[,k] <- log2(rowMeans(infRepsArray[,cond2,k]) + pc) -
                 log2(rowMeans(infRepsArray[,cond1,k]) + pc)
  }
  if (array) {
    return(diffs)
  }
  # median over inferential replicates
  rowMedians(diffs)
}

In [20]:
tse <- computeSizeFactors(tse, type="txp")
tse <- scaleInfReps(tse)
lfc <- getLog2FC(tse, colData(tse)[["condition"]])
descSize <- sapply(phangorn::Descendants(tree, seq(nrow(tse))), length)
metric <- abs(lfc)/mcols(tse)[['meanInfRV']]
objS <- solveForOptimalCut(tse, metVec = metric, type = 'max')
cuts[["LFC"]] <- objS[["cut"]]

Setting inf rep depth to mean depth



Progress:   1 on 100Progress:   2 on 100Progress:   3 on 100Progress:   4 on 100Progress:   5 on 100Progress:   6 on 100Progress:   7 on 100Progress:   8 on 100Progress:   9 on 100Progress:  10 on 100Progress:  11 on 100Progress:  12 on 100Progress:  13 on 100Progress:  14 on 100Progress:  15 on 100Progress:  16 on 100Progress:  17 on 100Progress:  18 on 100Progress:  19 on 100Progress:  20 on 100Progress:  21 on 100Progress:  22 on 100Progress:  23 on 100Progress:  24 on 100Progress:  25 on 100Progress:  26 on 100Progress:  27 on 100Progress:  28 on 100Progress:  29 on 100Progress:  30 on 100Progress:  31 on 100Progress:  32 on 100Progress:  33 on 100Progress:  34 on 100Progress:  35 on 100Progress:  36 on 100Progress:  37 on 100Progress:  38 on 100Progress:  39 on 100Progress:  40 on 100Progress:  41 on 100Progress:  42 on 100Progress:  43 on 100Progress:  44 on 100Progress:  45 on 100Progress:  46 on 100Progress:  47 on 100Progress:  48 on 100Progress:  49 on 100Progress:  50 on 100

In [21]:
y <- temp[sort(cuts[["LFC"]]),]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

detNodes[["LFC"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
    nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
    match(nodes, names(logFCNodes))
})
negNodes[["LFC"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
})

#### TPM LFC Desc

In [27]:
lfcTPM <- getScaledLFC(temp, "condition")
metric <- abs(lfcTPM)/mcols(tse)[['meanInfRV']]*descSize
objS <- solveForOptimalCut(tse, metVec = metric, type = 'max')
cuts[["lfcTPMDesc"]] <- objS[["cut"]]
y <- temp[sort(cuts[["lfcTPMDesc"]]),]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

detNodes[["lfcTPMDesc"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
    nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
    match(nodes, names(logFCNodes))
})
negNodes[["lfcTPMDesc"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })

#### TPM LFC

In [30]:
lfcTPM <- getScaledLFC(temp, "condition")
metric <- abs(lfcTPM)/mcols(tse)[['meanInfRV']]
objS <- solveForOptimalCut(tse, metVec = metric, type = 'max')
cuts[["lfcTPM"]] <- objS[["cut"]]
y <- temp[sort(cuts[["lfcTPM"]]),]
y <- fishpond::scaleInfReps(y)
y <- labelKeep(y)
set.seed(1)
y <- swish(y, x="condition")

detNodes[["lfcTPM"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
    nodes <- rownames(y)[which(mcols(y)[,"qvalue"] <= x)]
    match(nodes, names(logFCNodes))
})
negNodes[["lfcTPM"]] <- lapply(c(0.01, 0.05, 0.1), function(x) {
        nodes <- rownames(y)[setdiff(seq(nrow(y)), which(mcols(y)[,"qvalue"] <= x))]
        match(nodes, names(logFCNodes))
    })

In [20]:
df <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{
    print(n)
    stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], logFCNodes, tree = tree, negNodes = negNodes[[n]][[i]], lfcThresh = rootFC))
    colnames(stats) <- colnames(df)[c(3:5)]
    stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
    df <- rbind(df, stats)
}
print(df)

[1] "Swish"
[1] "tp 0.989603388525221"
[1] "tp 0.954755309325946"
[1] "tp 0.917580910586945"
[1] "irv_height_0.05"
[1] "tp 0.990189873417721"
[1] "tp 0.962002096436059"
[1] "tp 0.924371740161214"
[1] "irv_height_0.1"
[1] "tp 0.988809182209469"
[1] "tp 0.959091983920549"
[1] "tp 0.92020592020592"
[1] "irv_height_1"
[1] "tp 0.998265093684941"
[1] "tp 0.965827668565129"
[1] "tp 0.926988265971317"
[1] "LFC"
[1] "tp 0.988722950320024"
[1] "tp 0.959214501510574"
[1] "tp 0.914936823104693"
[1] "lfcTPMDesc"
[1] "tp 0.990125332320547"
[1] "tp 0.962094442659814"
[1] "tp 0.923234092235844"
[1] "lfcTPM"
[1] "tp 0.988598859885989"
[1] "tp 0.961711711711712"
[1] "tp 0.919837874352623"
              Method Metric    FDR_0.01   FDR_0.05   FDR_0.10
fdr            Swish    FDR  0.01039661 0.04524469 0.08241909
tpr            Swish    TPR   0.5841573  0.7050801  0.7604273
fdr1 irv_height_0.05    FDR 0.009810127  0.0379979 0.07562826
tpr1 irv_height_0.05    TPR   0.6166732  0.7234923  0.7684273
fdr2  irv_

In [33]:
saveDir <- "environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/comp_trees/DE"
save(cuts, file=file.path(saveDir, "cutsMean.RData"))
save(df, file=file.path(saveDir, "dfMean.RData"))

In [34]:
negNodesDesc <- list()
for(n in names(negNodes)) {
    if(n == "Swish")
        negNodesDesc[[n]] <- negNodes[[n]]
    if(n != "term")
        negNodesDesc[[n]] <- lapply(negNodes[[n]], function(x) unlist(Descendants(tree, x)))
}
negNodesDesc[["term"]] <- lapply(negNodes[["term"]], function(x) {
    dd <- x[x>l]
    r <- setdiff(x, dd)
    dd <- dd - l
    c(unlist(groupsClust[dd]), r) 
})

In [24]:
sapply(negNodes[["lfcTPM"]],length)
sapply(negNodesDesc[["lfcTPM"]],length)


[1] 83442 82779 82334

[1] 125497 123646 122411

In [37]:
dfO <- data.frame(Method=character(), Metric=character(), FDR_0.01 = numeric(), FDR_0.05 = numeric(), FDR_0.10 = numeric())
for(n in names(detNodes))
{
    print(n)
    if(n != "term") {
        stats <-  sapply(seq_along(detNodes[[n]]), function(i) computeMetOut(detNodes[[n]][[i]], 
                                                                             logFCNodes, tree = tree, 
                                                                             negNodes = negNodesDesc[[n]][[i]], 
                                                                             lfcThresh = rootFC))
        colnames(stats) <- colnames(df)[c(3:5)]
        stats <- cbind(Method=rep(n,2), Metric = c("FDR", "TPR"), stats)
        dfO <- rbind(dfO, stats)
    }
    
}
print(dfO)

[1] "Swish"
[1] "tp 0.990465071025491"
[1] "tp 0.962445003142678"
[1] "tp 0.926497404965633"
[1] "irv_height_0.05"
[1] "tp 0.990189873417721"
[1] "tp 0.962002096436059"
[1] "tp 0.924371740161214"
[1] "irv_height_0.1"
[1] "tp 0.988809182209469"
[1] "tp 0.959091983920549"
[1] "tp 0.92020592020592"
[1] "irv_height_1"
[1] "tp 0.998265093684941"
[1] "tp 0.965827668565129"
[1] "tp 0.926988265971317"
[1] "irv_height_5"
[1] "tp 0.990663488279698"
[1] "tp 0.957460418998881"
[1] "tp 0.922096520120292"
[1] "irv_height_10"
[1] "tp 0.990274265707061"
[1] "tp 0.955559020739124"
[1] "tp 0.919341334077589"
[1] "LFC"
[1] "tp 0.988722950320024"
[1] "tp 0.959214501510574"
[1] "tp 0.914936823104693"
[1] "lfcTPMDesc"
[1] "tp 0.990125332320547"
[1] "tp 0.962094442659814"
[1] "tp 0.923234092235844"
[1] "lfcTPM"
[1] "tp 0.988598859885989"
[1] "tp 0.961711711711712"
[1] "tp 0.919837874352623"
              Method Metric    FDR_0.01   FDR_0.05   FDR_0.10
fdr            Swish    FDR 0.009534929   0.037555  0.073

In [38]:
save(dfO, file=file.path(saveDir, "dfMeanO.RData"))